In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import json

## Constants

In [ ]:
IMG_RESOLUTION = (5120, 4096)

## Load the calibration data

In [ ]:
path_to_calib_data = '/home/deniskirbaba/Git/AITH-Hackathon-Severstal/src/calibration/data/calibration_data_real.json'

In [ ]:
with open(path_to_calib_data, 'r') as file:
    data = json.load(file)

calib_data = {}
for key, value in data.items():
    calib_data[key] = np.array(value)

for name, matrix in calib_data.items():
    print(f"Matrix {name}:\n{matrix}\n")

## Load the stereo image pair

* _cam1_ - левая камера
* _cam2_ - правая камера

In [ ]:
cam1_imgs_path = "data/cam2/"
cam2_imgs_path = "data/cam1/"
img_name = 'real_obj.jpg'

In [ ]:
cam1_img = cv.imread(cam1_imgs_path + img_name, cv.IMREAD_GRAYSCALE)
cam2_img = cv.imread(cam2_imgs_path + img_name, cv.IMREAD_GRAYSCALE)

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(cam1_img, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cam2_img, cmap='gray')
plt.axis('off')

plt.show()

## Rectification

In [ ]:
R1, R2, P1, P2, Q, validPixROI1, validPixROPI2 = cv.stereoRectify(calib_data['CM'], calib_data['dist'],
                                                calib_data['CM'], calib_data['dist'],
                                                IMG_RESOLUTION,
                                                calib_data['R'], calib_data['T'],
                                                flags=0, # cv.CALIB_ZERO_DISPARITY,
                                                alpha=-1
                                                )

In [ ]:
def show_rotation(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    singular = sy < 1e-6
    if not singular:
        yaw = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = np.arctan2(R[1, 0], R[0, 0])
    else:
        yaw = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = 0
    euler_angles_calculated = np.degrees(np.array([yaw, pitch, roll]))
    print(f"Yaw: {euler_angles_calculated[0]:.2f} degrees")
    print(f"Pitch: {euler_angles_calculated[1]:.2f} degrees")
    print(f"Roll: {euler_angles_calculated[2]:.2f} degrees")

In [ ]:
show_rotation(R1)
show_rotation(R2)

## Initialize undistort and rectification transform

In [ ]:
cam1_map1, cam1_map2 = cv.initUndistortRectifyMap(calib_data['CM'], calib_data['dist'], 
                           R1, P1, 
                           IMG_RESOLUTION, cv.CV_32FC1)

In [ ]:
cam2_map1, cam2_map2 = cv.initUndistortRectifyMap(calib_data['CM'], calib_data['dist'], 
                           R2, P2, 
                           IMG_RESOLUTION, cv.CV_32FC1)

In [ ]:
def draw_epipolar_lines(img1, img2, color=255, step=250):
    img1_lines = img1.copy()
    img2_lines = img2.copy()

    height = img1.shape[0]

    for y in range(0, height, step):
        cv.line(img1_lines, (0, y), (img1.shape[1], y), color, 7)
        cv.line(img2_lines, (0, y), (img2.shape[1], y), color, 7)

    return img1_lines, img2_lines

In [ ]:
rectified_cam1_img = cv.remap(cam1_img, cam1_map1, cam1_map2, interpolation=cv.INTER_LINEAR)
rectified_cam2_img = cv.remap(cam2_img, cam2_map1, cam2_map2, interpolation=cv.INTER_LINEAR)

epipolar_img_left, epipolar_img_right = draw_epipolar_lines(rectified_cam1_img, rectified_cam2_img)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(epipolar_img_left, cmap='gray')
plt.title('Rectified Left Camera Image with Epipolar Lines')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(epipolar_img_right, cmap='gray')
plt.title('Rectified Right Camera Image with Epipolar Lines')
plt.axis('off')

plt.show()

Эпиполярные линии верны, а значит что найденные внутренние и внешние параметры также верны.   
Значит мы можем произвести ректификацию изображений, а значит можем и использовать алгоритм BM, SGBM для вычисления диспаритетов.

## Apply bluring to reduce noise and improve matching accuracy

In [ ]:
preprocessed_left_img = cv.GaussianBlur(rectified_cam1_img, (5, 5), 0)
preprocessed_right_img = cv.GaussianBlur(rectified_cam2_img, (5, 5), 0)

## Вычисление диспаритетов

### С помощью SGMB

In [ ]:
# SGBM Parameters
min_disp = -1  # Minimum disparity (typically 0 or a small negative number)
num_disp = 12 * 16  # Number of disparities to search (must be divisible by 16)
block_size = 13  # Block size to match (the size of the windows for matching)

stereo = cv.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 1 * block_size ** 2,  # Smoothness penalty (smaller P1)
    P2=32 * 1 * block_size ** 2,  # Smoothness penalty (larger P2)
    disp12MaxDiff=3,  # Maximum allowed difference in the left-right disparity check
    uniquenessRatio=10,  # Margin in percentage by which the best cost function value should "win"
    speckleWindowSize=100,  # Maximum size of smooth disparity regions to consider them noise
    speckleRange=25,  # Maximum disparity variation within each connected component
    preFilterCap=50  # Pre-filtering before disparity computation
)

disparity = stereo.compute(preprocessed_left_img, preprocessed_right_img).astype(np.float32) / 16.0

In [ ]:
disparity_normalized = cv.normalize(disparity, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
disparity_normalized = np.uint8(disparity_normalized)

plt.imshow(disparity_normalized, 'gray')
plt.axis('off')
plt.show()

### С помощью BM

In [ ]:
# StereoBM parameters
num_disp = 6 * 16  # Number of disparities to search (must be divisible by 16)
block_size = 9  # Block size to match (should be an odd number, typically between 5 and 21)

stereo_bm = cv.StereoBM_create(numDisparities=num_disp, blockSize=block_size)

# StereoBM works best with images that have been pre-processed by cv.equalizeHist
preprocessed_left_img = cv.equalizeHist(preprocessed_left_img)
preprocessed_right_img = cv.equalizeHist(preprocessed_right_img)

disparity = stereo_bm.compute(preprocessed_left_img, preprocessed_right_img).astype(np.float32) / 16.0

In [ ]:
disparity_normalized = cv.normalize(disparity, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
disparity_normalized = np.uint8(disparity_normalized)

plt.imshow(disparity_normalized, cmap='gray')
plt.axis('off')
plt.show()

## Вычисление points cloud

In [ ]:
points = cv.reprojectImageTo3D(disparity_normalized, Q)

gray_image = cam1_img

# Filter points with valid disparities (mask out points with no disparity)
mask = disparity_normalized > disparity_normalized.min()
out_points = points[mask]
out_intensities = gray_image[mask]

out_colors = out_intensities / 255.0

In [ ]:
points.shape, out_points.shape

In [ ]:
# Print ranges of x, y, z

x_min, x_max = np.min(out_points[:, 0]), np.max(out_points[:, 0])
y_min, y_max = np.min(out_points[:, 1]), np.max(out_points[:, 1])
z_min, z_max = np.min(out_points[:, 2]), np.max(out_points[:, 2])

print(f"x range: ({x_min}, {x_max})")
print(f"y range: ({y_min}, {y_max})")
print(f"z range: ({z_min}, {z_max})")

In [ ]:
# Filter points based on X, Y or Z axis (e.g., filtering out distant points)

# idx = np.abs(out_points[:, 0]) < 50
# out_points = out_points[idx]
# out_intensities = out_intensities[idx]

# idx = np.abs(out_points[:, 1]) < 50
# out_points = out_points[idx]
# out_intensities = out_intensities[idx]

# idx = np.abs(out_points[:, 2]) < 50
# out_points = out_points[idx]
# out_intensities = out_intensities[idx]

In [ ]:
points.shape, out_points.shape

In [ ]:
def write_ply(fn, verts, intensities):
    ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
    '''
    
    verts = verts.reshape(-1, 3)

    intensities = intensities.reshape(-1, 1) 
    intensities = np.clip(intensities, 0, 255)  
    out_colors = np.hstack([intensities, intensities, intensities]).astype(np.uint8)

    verts_with_colors = np.hstack([verts, out_colors])

    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts_with_colors, fmt='%f %f %f %d %d %d')

write_ply('synthetic_obj_bm.ply', out_points, out_intensities)